In [1]:
from sklearn.model_selection import train_test_split
dataset_path = "../data/science_2021_11_28.txt"

train, test = train_test_split(open(dataset_path).readlines(), test_size=0.15) 
def build_text_files(data, dest_path):
    f = open(dest_path, 'w')
    f.write("".join([e for e in data if e != '\n']))

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'
build_text_files(train, train_path)
build_text_files(test, test_path)

print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 38736
Test dataset length: 6836


In [2]:
!head -n 10 train_dataset.txt

age,artificial intelligence,biology ### What is the name of the first artificial human?
transhumanism ### Which is the material using for folding wings in the real robots arm ?
knowledge,science ### Which is the most important topic in this course ?
physic ### Given that it has been accepted, with or without evidence, that universals are "empirically real", then it seems to follow, does it not?
physic ###  Is one mind different from other minds if they're based on a similar physical body... Why do you think so?
transhumanism,religion ### Do you believe that transhumanism will occur?
knowledge,philosophy ### What did you looked up to find out?
science ### Is there a way to know if something is entirely positive?
artificial intelligence,philosophy ###  Do you see any possible risks in the development of transhumanist AI?
Artificial intelligence,science ### What is the difference between reality and a simulation?


In [3]:
from transformers import AutoTokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path,test_path,tokenizer)


from transformers import Trainer, TrainingArguments,AutoModelForCausalLM
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name)

/home/louis/Documents/langame-worker/env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
import wandb
wandb.init(project="my-test-project", entity="louis030195")
model.half()
training_args = TrainingArguments(
    output_dir="./gpt2-cw",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    eval_steps = 400,
    save_steps=800, 
    warmup_steps=500,
    prediction_loss_only=True,
    report_to="wandb"
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

PyTorch: setting up devices
***** Running training *****
  Num examples = 7828
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 369
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
saved_model = "gpt2_langame_science"
trainer.save_model(output_dir=saved_model)
# !zip -r "${saved_model}.zip" $saved_model
from transformers import pipeline

gen = pipeline('text-generation', model=saved_model, tokenizer=model_name)
gen("hey")

Saving model checkpoint to gpt2_langame_science
Configuration saved in gpt2_langame_science/config.json
Model weights saved in gpt2_langame_science/pytorch_model.bin
loading configuration file gpt2_langame_science/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2_langame_science",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0